<a href="https://colab.research.google.com/github/jacksonkarel/cacique/blob/main/notebooks/cacique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.ipynb), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [ ]:
!pip install langchain openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp drive/MyDrive/Datasets/* .

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import os
import json
keys = json.load(open('keys.json'))
os.environ["OPENAI_API_KEY"] = keys['openai']
os.environ["SERPAPI_API_KEY"] = keys['serp']

In [ ]:
!rm keys.json

In [ ]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.experimental import BabyAGI

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [ ]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [ ]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import PythonREPL

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
python_repl = PythonREPL()
tools = [
    # Tool(
    #     name="Search",
    #     func=search.run,
    #     description="useful for when you need to answer questions about current events",
    # ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you expect output it should be printed out."
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [ ]:
import pandas as pd
df = pd.read_csv("LivingPlanetIndex2022_public.csv")
print(list(df.columns))

In [ ]:
OBJECTIVE = "Each row is a different species, give a list of species that match the Location the Region of the mouth of the Karipunas River in the Madeira River in the LivingPlanetIndex2022_public.csv dataset. Summarize the information for a biodiversity expert"

In [ ]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, task_execution_chain=agent_executor, verbose=verbose, max_iterations=max_iterations
)

In [ ]:
baby_agi({"objective": OBJECTIVE})

In [ ]:
OBJECTIVE = "Find the answer to the question how many of each of these species were observed in column 2022?"

In [ ]:
max_iterations: Optional[int] = 1
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, task_execution_chain=agent_executor, verbose=verbose, max_iterations=max_iterations
)

In [ ]:
baby_agi({"objective": OBJECTIVE})